## Create a Bing Grounded Agent

https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-agents/samples/agents_tools/sample_agents_bing_grounding.py

In [ ]:
from dotenv import load_dotenv  
import os  
# Configure logger
import logging
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.agents.models import BingGroundingTool

# Load the .env file  
load_dotenv(override=True) 

# Now you can use the environment variables, for example:  
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model=os.environ["AZURE_OPENAI_GPT_MODEL"]


BING_CONNECTION_NAME = os.environ["BING_CONNECTION_NAME"]
print(f"BING_CONNECTION_NAME: {BING_CONNECTION_NAME}")

In [ ]:
# Create a logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

# Clear existing handlers to avoid duplicate logs
if logger.hasHandlers():
    logger.handlers.clear()

# Create handler that outputs to notebook
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)

# Example usage
logger.info("Logger is now printing to the notebook!")

In [ ]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]


# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()

if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)

In [ ]:
bing_connection = project_client.connections.get(name=BING_CONNECTION_NAME)
logger.info(f"bing_connection: {bing_connection['name']}")
conn_id = bing_connection.id

bing = BingGroundingTool(connection_id=conn_id)

In [ ]:

agent_name = os.environ["AZURE_AI_AGENT_NAME_02"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")
else:
    logger.info(f"Agent '{agent_name}' found in the project.")
    logger.info(f"Agent ID: {agent.id}")

In [ ]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

if not agent:
    logger.info(f"Agent with name '{agent_name}' not found, creating a new agent.")

    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Specify the model to use
        name=agent_name,  # Name of the agent
        instructions="You are a helpful agent with a tool for grounding with bing, this this tool to find relevant answers",  # Instructions for the agent
        tools=bing.definitions,  # Attach the Bing Grounding tool
    )
    logger.info(f"Created agent, ID: {agent.id}")
else:
    logger.info(f"Found agent by name '{agent_name}', ID={agent.id}")

In [ ]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="what is the weather in London?",  # Message content
    )
print(f"Created message, ID: {message['id']}")

In [ ]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

print(agent.name)
print(agent.id)

In [ ]:
from azure.ai.agents.models import MessageRole

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)

for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                bing_grounding_details = call.get("bing_grounding", {})
                if bing_grounding_details:
                    print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

        print()  # add an extra newline between steps

for message in messages:
    if message.text_messages:
        print(f"{message.role}: {message.text_messages[-1].text.value}")
        if message.role == MessageRole.AGENT:
            for annotation in message.url_citation_annotations:
                print(f"    URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")